# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 15 2022 10:09AM,244139,10,PRF-35076,Bio-PRF,Released
1,Aug 15 2022 10:09AM,244139,10,PRF-35080,Bio-PRF,Released
2,Aug 15 2022 10:09AM,244139,10,PRF-35083,Bio-PRF,Released
3,Aug 15 2022 10:09AM,244139,10,PRF-35086,Bio-PRF,Released
4,Aug 15 2022 10:09AM,244139,10,PRF-35110,Bio-PRF,Released
5,Aug 15 2022 10:09AM,244139,10,PRF-35123,Bio-PRF,Released
6,Aug 15 2022 10:09AM,244139,10,PRF-35127,Bio-PRF,Released
7,Aug 15 2022 10:09AM,244139,10,PRF-35134,Bio-PRF,Released
8,Aug 15 2022 10:08AM,244138,10,8397650,Eywa Pharma Inc.,Released
9,Aug 15 2022 10:08AM,244138,10,8404750,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,244133,Released,121
15,244135,Released,10
16,244137,Released,1
17,244138,Released,6
18,244139,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244133,NaN,121.0
244135,NaN,10.0
244137,NaN,1.0
244138,NaN,6.0
244139,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244108,6.0,5.0
244109,0.0,27.0
244111,0.0,1.0
244118,0.0,25.0
244119,0.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244108,6,5
244109,0,27
244111,0,1
244118,0,25
244119,0,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244108,6,5
1,244109,0,27
2,244111,0,1
3,244118,0,25
4,244119,0,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244108,6,5
1,244109,,27
2,244111,,1
3,244118,,25
4,244119,,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 15 2022 10:09AM,244139,10,Bio-PRF
8,Aug 15 2022 10:08AM,244138,10,Eywa Pharma Inc.
14,Aug 15 2022 9:43AM,244137,16,"SHL Pharma, LLC"
15,Aug 15 2022 9:28AM,244135,10,ISDIN Corporation
25,Aug 15 2022 9:22AM,244133,10,ISDIN Corporation
146,Aug 15 2022 9:01AM,244132,12,HealthCaps LLC
147,Aug 15 2022 8:58AM,244131,10,Beach Patient Assistance
151,Aug 15 2022 8:52AM,244130,16,Sartorius Stedim North America
152,Aug 15 2022 8:49AM,244129,10,Bio-PRF
153,Aug 15 2022 8:37AM,244127,10,Beach Products Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 15 2022 10:09AM,244139,10,Bio-PRF,,8
1,Aug 15 2022 10:08AM,244138,10,Eywa Pharma Inc.,,6
2,Aug 15 2022 9:43AM,244137,16,"SHL Pharma, LLC",,1
3,Aug 15 2022 9:28AM,244135,10,ISDIN Corporation,,10
4,Aug 15 2022 9:22AM,244133,10,ISDIN Corporation,,121
5,Aug 15 2022 9:01AM,244132,12,HealthCaps LLC,1,
6,Aug 15 2022 8:58AM,244131,10,Beach Patient Assistance,,4
7,Aug 15 2022 8:52AM,244130,16,Sartorius Stedim North America,1,
8,Aug 15 2022 8:49AM,244129,10,Bio-PRF,,1
9,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 10:09AM,244139,10,Bio-PRF,8,
1,Aug 15 2022 10:08AM,244138,10,Eywa Pharma Inc.,6,
2,Aug 15 2022 9:43AM,244137,16,"SHL Pharma, LLC",1,
3,Aug 15 2022 9:28AM,244135,10,ISDIN Corporation,10,
4,Aug 15 2022 9:22AM,244133,10,ISDIN Corporation,121,
5,Aug 15 2022 9:01AM,244132,12,HealthCaps LLC,,1
6,Aug 15 2022 8:58AM,244131,10,Beach Patient Assistance,4,
7,Aug 15 2022 8:52AM,244130,16,Sartorius Stedim North America,,1
8,Aug 15 2022 8:49AM,244129,10,Bio-PRF,1,
9,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 10:09AM,244139,10,Bio-PRF,8,
1,Aug 15 2022 10:08AM,244138,10,Eywa Pharma Inc.,6,
2,Aug 15 2022 9:43AM,244137,16,"SHL Pharma, LLC",1,
3,Aug 15 2022 9:28AM,244135,10,ISDIN Corporation,10,
4,Aug 15 2022 9:22AM,244133,10,ISDIN Corporation,121,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 10:09AM,244139,10,Bio-PRF,8.0,NaN
1,Aug 15 2022 10:08AM,244138,10,Eywa Pharma Inc.,6.0,NaN
2,Aug 15 2022 9:43AM,244137,16,"SHL Pharma, LLC",1.0,NaN
3,Aug 15 2022 9:28AM,244135,10,ISDIN Corporation,10.0,NaN
4,Aug 15 2022 9:22AM,244133,10,ISDIN Corporation,121.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 10:09AM,244139,10,Bio-PRF,8.0,0.0
1,Aug 15 2022 10:08AM,244138,10,Eywa Pharma Inc.,6.0,0.0
2,Aug 15 2022 9:43AM,244137,16,"SHL Pharma, LLC",1.0,0.0
3,Aug 15 2022 9:28AM,244135,10,ISDIN Corporation,10.0,0.0
4,Aug 15 2022 9:22AM,244133,10,ISDIN Corporation,121.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2929504,207.0,6.0
12,732369,36.0,1.0
16,488267,1.0,1.0
20,244122,6.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2929504,207.0,6.0
1,12,732369,36.0,1.0
2,16,488267,1.0,1.0
3,20,244122,6.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,207.0,6.0
1,12,36.0,1.0
2,16,1.0,1.0
3,20,6.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,207.0
1,12,Released,36.0
2,16,Released,1.0
3,20,Released,6.0
4,10,Executing,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,20
Status,,,,
Executing,6.0,1.0,1.0,0.0
Released,207.0,36.0,1.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,20
0,Executing,6.0,1.0,1.0,0.0
1,Released,207.0,36.0,1.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,20
0,Executing,6.0,1.0,1.0,0.0
1,Released,207.0,36.0,1.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()